In [12]:
import matplotlib.pyplot as plt
from soilgrids import SoilGrids
import geopandas as gpd
import os
import subprocess as subp
import time
from osgeo import gdal, gdal_array

In [40]:
geojson = '../../../../src/data_preprocessing/base_geojson/TL_state_shapefile_for_clip.geojson'
gdf = gpd.read_file(geojson)
gdf = gdf.to_crs(3857) # because Soilgrid data is in EPSG:3857
bnd  = gdf.bounds #get corner points

In [17]:
os.mkdir('soil') #create directory to save file

SOC

In [18]:
# get data from SoilGrids
soil_grids = SoilGrids()
data = soil_grids.get_coverage_data(service_id='soc', coverage_id='soc_0-5cm_mean', 
                                    #height=2000,width=2000,
                                       west=bnd.minx[0], south=bnd.miny[0], east=bnd.maxx[0], north=bnd.maxy[0],  
                                       crs='urn:ogc:def:crs:EPSG::3857',output='soil/soc_0-5cm_mean_telangana.tif')

# show metadata
for key, value in soil_grids.metadata.items():
    print('{}: {}'.format(key,value))


variable_name: Soil organic carbon content
variable_units: dg/kg
service_url: https://maps.isric.org/mapserv?map=/map/soc.map
service_id: soc
coverage_id: soc_0-5cm_mean
crs: urn:ogc:def:crs:EPSG::3857
bounding_box: (8598011.842434999, 1785788.1580495879, 9051922.07251603, 2263189.1948075155)
grid_res: [250, 250]


CLay

In [19]:
# get data from SoilGrids
soil_grids = SoilGrids()
data = soil_grids.get_coverage_data(service_id='clay', coverage_id='clay_0-5cm_mean', 
                                    #height=2000,width=2000,
                                       west=bnd.minx[0], south=bnd.miny[0], east=bnd.maxx[0], north=bnd.maxy[0],  
                                       crs='urn:ogc:def:crs:EPSG::3857',output='soil/clay_0-5cm_mean_telangana.tif')

# show metadata
for key, value in soil_grids.metadata.items():
    print('{}: {}'.format(key,value))

variable_name: Clay content
variable_units: g/kg
service_url: https://maps.isric.org/mapserv?map=/map/clay.map
service_id: clay
coverage_id: clay_0-5cm_mean
crs: urn:ogc:def:crs:EPSG::3857
bounding_box: (8598011.842434999, 1785788.1580495879, 9051922.07251603, 2263189.1948075155)
grid_res: [250, 250]


Bulk Density

In [20]:
# get data from SoilGrids
soil_grids = SoilGrids()
data = soil_grids.get_coverage_data(service_id='bdod', coverage_id='bdod_0-5cm_mean', 
                                    #height=2000,width=2000,
                                       west=bnd.minx[0], south=bnd.miny[0], east=bnd.maxx[0], north=bnd.maxy[0],  
                                       crs='urn:ogc:def:crs:EPSG::3857',output='soil/bdod_0-5cm_mean_telangana.tif')

# show metadata
for key, value in soil_grids.metadata.items():
    print('{}: {}'.format(key,value))

variable_name: Bulk density
variable_units: cg/cm3
service_url: https://maps.isric.org/mapserv?map=/map/bdod.map
service_id: bdod
coverage_id: bdod_0-5cm_mean
crs: urn:ogc:def:crs:EPSG::3857
bounding_box: (8598011.842434999, 1785788.1580495879, 9051922.07251603, 2263189.1948075155)
grid_res: [250, 250]


Reproject, Clip and create COG

In [21]:
basepath='soil/'

if 'projected' not in os.listdir():
    os.mkdir('projected')
else:
    print('directory exists')

arr = os.listdir(basepath)
for i in arr:
    cmd="gdalwarp -of GTIFF  -r cubic -t_srs '+proj=longlat +datum=WGS84 +no_defs'"+" "+basepath+str(i)+" projected/"+str(i)
    print(cmd)
    try:
        subp.check_call(str(cmd), shell=True)
    except:
        print("end")
    time.sleep(1)

basepath = 'projected/'
# Open band 1 as array
arr = os.listdir(basepath)

if 'clipped' not in os.listdir():
    os.mkdir('clipped')
else:
    print('directory exists')
    
for i in arr:
    cmd="gdalwarp -dstnodata -9999 -cutline "+geojson+" -crop_to_cutline "+basepath+str(i)+" clipped/"+i
    print(cmd)
    try:
        subp.check_call(str(cmd), shell=True)
    except:
        print("end")
    time.sleep(1)


arr = os.listdir('clipped/')

if 'cog' not in os.listdir():
    os.mkdir('cog')
else:
    print('directory exists')

for i in arr:
    cmd="gdal_translate clipped/"+str(i)+" cog/"+str(i)+" -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 -co TILED=YES"
    print(cmd)
    try:
        subp.check_call(str(cmd), shell=True)
    except:
        print("end")
    time.sleep(1)

cmd="rm -r soil clipped projected"
print(cmd)
try:
    subp.check_call(str(cmd), shell=True)
except:
    print("end")

directory exists
gdalwarp -of GTIFF  -r cubic -t_srs '+proj=longlat +datum=WGS84 +no_defs' soil/soc_0-5cm_mean_telangana.tif projected/soc_0-5cm_mean_telangana.tif
Creating output file that is 1863P x 1864L.
Processing soil/soc_0-5cm_mean_telangana.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -of GTIFF  -r cubic -t_srs '+proj=longlat +datum=WGS84 +no_defs' soil/clay_0-5cm_mean_telangana.tif projected/clay_0-5cm_mean_telangana.tif
Creating output file that is 1863P x 1864L.
Processing soil/clay_0-5cm_mean_telangana.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -of GTIFF  -r cubic -t_srs '+proj=longlat +datum=WGS84 +no_defs' soil/bdod_0-5cm_mean_telangana.tif projected/bdod_0-5cm_mean_telangana.tif
Creating output file that is 1863P x 1864L.
Processing soil/bdod_0-5cm_mean_telangana.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -dstnodata -9999 -cutline TL_state_shapefile_for_clip